In [1]:
import os
import sys
import pickle

import numpy as np
import matplotlib.pyplot as plt

from utils import *
from AnalyseDefects import AnalyseDefects

plt.style.use('sg_article')

In [48]:
path = f'C:\\Users\\Simon Andersen\\Documents\\Uni\\Speciale\\Hyperuniformity\\na512l_cl'
d_cluster = dict(path = path, \
              suffix = "cluster", priority = -1, LX = 512, Nframes = 400)
output_path = 'data\\nematic_analysis512_LL0.05\\cluster' 
if not os.path.exists(output_path):
    os.makedirs(output_path)

Nexp_list = []
act_list = []
act_dir_list = []

for i, dir in enumerate(os.listdir(path)):
    if dir.startswith('analysis'):
        act_list.append(float(dir.split('_')[-1]))
        act_dir_list.append(os.path.join(path, dir))
for i, file in enumerate(os.listdir(os.path.join(path, os.listdir(path)[0]))):
    print(file)
    if file.startswith('zeta'):
        Nexp_list.append(int(file.split('_')[-1]))


zeta_0.019_counter_0
zeta_0.019_counter_1
zeta_0.019_counter_2
zeta_0.019_counter_3
zeta_0.019_counter_4
zeta_0.019_counter_5
zeta_0.019_counter_6
zeta_0.019_counter_7
zeta_0.019_counter_8
zeta_0.019_counter_9


In [50]:
act_dir_list, act_list

(['C:\\Users\\Simon Andersen\\Documents\\Uni\\Speciale\\Hyperuniformity\\na512l_cl\\analysis_zeta_0.019',
  'C:\\Users\\Simon Andersen\\Documents\\Uni\\Speciale\\Hyperuniformity\\na512l_cl\\analysis_zeta_0.0225',
  'C:\\Users\\Simon Andersen\\Documents\\Uni\\Speciale\\Hyperuniformity\\na512l_cl\\analysis_zeta_0.023',
  'C:\\Users\\Simon Andersen\\Documents\\Uni\\Speciale\\Hyperuniformity\\na512l_cl\\analysis_zeta_0.0235',
  'C:\\Users\\Simon Andersen\\Documents\\Uni\\Speciale\\Hyperuniformity\\na512l_cl\\analysis_zeta_0.024'],
 [0.019, 0.0225, 0.023, 0.0235, 0.024])

In [40]:
act_idx = -1
exp_idx = 0
Nframes = d_cluster['Nframes']
Nexp = len(Nexp_list)
Nact = len(act_list)

pt = os.path.join(path, os.listdir(path)[act_idx], f'zeta_{act_list[act_idx]}_counter_{Nexp_list[exp_idx]}')

In [5]:
dirr = [os.path.join(path, os.listdir(path)[i], f'zeta_{act_list[]}_counter_{Nexp_list[exp_idx]}')]

'C:\\Users\\Simon Andersen\\Documents\\Uni\\Speciale\\Hyperuniformity\\na512l_cl\\analysis_zeta_0.024\\zeta_0.024_counter_0'

In [6]:
with open(os.path.join(pt, 'defect_positions.pkl'), 'rb') as f:
        defect_positions = pickle.load(f)
with open(os.path.join(pt, 'labels_rm33.pkl'), 'rb') as f:
        labels = pickle.load(f)

cluster_arr = np.nan * np.zeros([Nframes, 4, Nact, Nexp])
Ncmin = 0

In [28]:
la = labels[0]

min_cluster_size = 2

unique, counts = np.unique(la, return_counts=True)
# Only count clusters with more than min_cluster_size defects
mask = (counts >= min_cluster_size)

cluster_sizes, cluster_counts = np.unique(counts[mask], return_counts=True)


Ndefects_tot = counts.sum()
Ndefects_unclustered = counts[~mask].sum()
Ndefects_clustered = counts[mask].sum()
Nclusters = len(cluster_sizes)
cluster_size_av = np.mean(counts[mask])
cluster_size_std = np.std(counts[mask])

print(f'Number of clustered defects: {Ndefects_clustered}')
print(f'Number of unclustered defects: {Ndefects_unclustered}')
print(f'Number of clusters: {Nclusters}')
print(f'Averge number of defects per cluster: {np.mean(counts):.2f} +/- {np.std(counts):.2f}')

Number of clustered defects: 424
Number of unclustered defects: 2
Number of clusters: 5
Averge number of defects per cluster: 60.86 +/- 96.19


In [41]:
with open(os.path.join(pt, 'defect_positions.pkl'), 'rb') as f:
        defect_positions = pickle.load(f)
with open(os.path.join(pt, 'labels_rm33.pkl'), 'rb') as f:
        labels = pickle.load(f)


cluster_arr = np.nan * np.zeros([Nframes, 5, Nact, Nexp])
Ncmin = 2

for j, frame in enumerate(labels[:Nframes]):

        # store the number of defects 
        cluster_arr[j, 0, act_idx, exp_idx] = len(frame)

        unique, counts = np.unique(frame, return_counts=True)

        # Only count clusters with more than Ncmin defects
        mask = (counts >= Ncmin)

        counts_above_min = counts[mask]

        # Find the sizes of the clusters and the number of clusters of each size
        cluster_sizes, cluster_counts = np.unique(counts_above_min, return_counts=True)

        # store the number of clustered defects
        cluster_arr[j, 1, act_idx, exp_idx] = counts_above_min.sum()

        # store the number of clusters
        cluster_arr[j, 2, act_idx, exp_idx] = len(cluster_sizes)
   
        # store the average cluster size
        cluster_arr[j, 3, act_idx, exp_idx] = np.mean(counts_above_min)

        # store the standard deviation of the cluster sizes
        cluster_arr[j, 4, act_idx, exp_idx] = np.std(counts_above_min)




# store Ndefects_tot, Ndefects_clusterd, Nclust, AvgSize, StidSize for each frame and each activity and each experiment
# something with probability of belonging to a cluster up to N=20 or something?
# excluding singles (but counting them) and possibly also pairs

In [45]:
avv = cluster_arr[:, :, act_idx, exp_idx].mean(axis=0)
avv[-1] /= np.sqrt(Nframes)
avv

array([423.5875    , 420.5225    ,   3.895     , 123.28929466,
         7.59574666])

In [54]:

def extract_results(Nframes, act_list, act_dir_list, Nexp, Ncmin=2, save = True,):
    """
    Analyse the defects for all the input folders
    """
    for N in range(1):

        # create arrays to store the clustering data
        cluster_arr = np.nan * np.zeros([Nframes, 5, len(act_list), Nexp])
        
      #  print('Analysing clustering data for input folder {}'.format(self.input_paths[N]))
        for i, (act, act_dir) in enumerate(zip(act_list, act_dir_list)):

            exp_list = []
            exp_dir_list = []

            for file in os.listdir(act_dir):
                exp_count = file.split('_')[-1]
                exp_list.append(int(exp_count))
                exp_dir_list.append(os.path.join(act_dir, file))

            # sort the activity list and the activity directory list
            exp_list, exp_dir_list = zip(*sorted(zip(exp_list, exp_dir_list)))

            for j, (exp, exp_dir) in enumerate(zip(exp_list, exp_dir_list)):


           #     with open(os.path.join(exp_dir, 'defect_positions.pkl'), 'rb') as f:
            #            defect_positions = pickle.load(f)
                with open(os.path.join(exp_dir, 'labels_rm33.pkl'), 'rb') as f:
                        labels = pickle.load(f)


                for j, frame in enumerate(labels[:Nframes]):

                        # store the number of defects 
                        cluster_arr[j, 0, act_idx, exp_idx] = len(frame)

                        unique, counts = np.unique(frame, return_counts=True)

                        # Only count clusters with more than Ncmin defects
                        mask = (counts >= Ncmin)

                        counts_above_min = counts[mask]

                        # Find the sizes of the clusters and the number of clusters of each size
                        cluster_sizes, cluster_counts = np.unique(counts_above_min, return_counts=True)

                        # store the number of clustered defects
                        cluster_arr[j, 1, act_idx, exp_idx] = counts_above_min.sum()

                        # store the number of clusters
                        cluster_arr[j, 2, act_idx, exp_idx] = len(cluster_sizes)
                
                        # store the average cluster size
                        cluster_arr[j, 3, act_idx, exp_idx] = np.mean(counts_above_min)

                        # store the standard deviation of the cluster sizes
                        cluster_arr[j, 4, act_idx, exp_idx] = np.std(counts_above_min)

        if save:
            pass
    return cluster_arr

In [55]:
cl_arr = extract_results(Nframes, act_list, act_dir_list, Nexp, Ncmin=2, save=False)

c:\Users\Simon Andersen\miniconda3\envs\sf\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Simon Andersen\miniconda3\envs\sf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\Simon Andersen\miniconda3\envs\sf\lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\Simon Andersen\miniconda3\envs\sf\lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\Simon Andersen\miniconda3\envs\sf\lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


TypeError: object of type 'NoneType' has no len()